In [1]:
import sys
sys.path.append("./expectation_value")

from expectationvalue import ExpVal
import numpy as np
import qiskit.quantum_info as qif
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

In [2]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver


class MoleculeHamiltonian():
    """
    Filler

    """
    
    def __init__(self,
                 molecule,
                 converter,
                 basis = 'sto3g',
                ):
        """

        Parameters
        ----------
        molecule : string.
            
        converter : QubitConverter.
            
        basis : string.
            

        """
        
        self.molecule = molecule
        self.basis = basis
        driver = PySCFDriver(
            atom= self.molecule,
            basis=self.basis,
            charge=0,
            spin=0,
            unit=DistanceUnit.ANGSTROM,
            )                                  # Define the PySCFDriver for a given molecule
        
        self.driver = driver
        self.problem = driver.run()            # Define the ElectronicStructureProblem from the driver
        self.converter = converter
        
    def Hamiltonian(self,
                    freeze_core = False,
                    remove_orbitals = None):
        
        
        """
        Filler


        Parameters
        ----------
        freeze_core : bool.
            
        remove_orbitals : List, optional.
            
            
        Returns
        -------
        qubit_op : PauliSumOp.
            
        """
        problem = self.problem
        fermionic_op = self.problem.hamiltonian.second_q_op()
            
        if freeze_core:
            fc_transformer = FreezeCoreTransformer(freeze_core=freeze_core, remove_orbitals=remove_orbitals)
            problem = fc_transformer.transform(problem)
            fermionic_op = problem.hamiltonian.second_q_op()
            
        qubit_op = self.converter.convert(fermionic_op,
                                           sector_locator=problem.symmetry_sector_locator)
        return qubit_op
    
    def ComputeGroundState(self , circuit = True):
        
        """
        Filler.


        Parameters
        ----------
        circuit : bool.
            


        Returns
        -------
        state : QuantumCircuit or array(2**num_qubits)
            
        eigenvalue : float.
            
        """
        solver = NumPyMinimumEigensolver()
        
        ground_solver = GroundStateEigensolver(self.converter, solver)
        ground_state = ground_solver.solve(self.problem)
        
        state = ground_state.groundstate[0]
        eigenvalue = ground_state.groundenergy
        
        if circuit:
            return state, eigenvalue
        else:
            return qif.Statevector(state).data, eigenvalue

In [3]:
def get_number_nbody_terms(hamiltonian):
    """
    Gets the needed data of the hamiltonian to compute the expectation values using ExpVal.exp_val().


    Parameters
    ----------
    hamiltonian : PauliSumOp.
        Hamiltonian.


    Returns
    -------
    bodies : list.
        N-bodies interactions.
    """
    
    n_bodies_inter = []
    
    for i in range(len(hamiltonian)):
        pauli_string = hamiltonian[i].to_pauli_op().primitive.to_label()
        num_x = pauli_string.count('X')
        num_y = pauli_string.count('Y')
        
        n_body_terms = num_x+num_y
        
        n_bodies_inter.append(n_body_terms)
    
    bodies = list(set(n_bodies_inter))
    
    return bodies


def get_obs_data(hamiltonian):
    """
    Gets the needed data of the hamiltonian to compute the expectation values using ExpVal.exp_val().


    Parameters
    ----------
    hamiltonian : PauliSumOp.
        Hamiltonian.


    Returns
    -------
    coeffs : array(num_paulis).
        Coefficients of each pauli string of the hamiltonian.
    obs : array(2, 2, n_qubits, num_paulis)
        Observable made of every pauli string of the hamiltonian.
    """
    
    num_paulis = len(hamiltonian)
    n_qubits = hamiltonian.num_qubits

    paulis_dict = {'X': np.array([[0., 1.], [1., 0.]], dtype="complex"),
               'Y': np.array([[0., -1.*1j], [1.*1j, 0.]], dtype="complex"),
               'Z': np.array([[1., 0], [0, -1.]], dtype="complex"),
               'I': np.array([[1., 0], [0, 1.]], dtype="complex")}

    obs = np.zeros((2, 2, n_qubits, num_paulis), dtype=complex)
    coeffs = np.zeros(num_paulis, dtype=complex)

    for i in range(num_paulis):
        pauli_string_coeff = hamiltonian[i].coeffs[0]
        pauli_string = hamiltonian[i].to_pauli_op().primitive.to_label()
        pauli_string_list = [paulis_dict[i] for i in pauli_string]
        obs[:,:,:,i] =  np.stack(pauli_string_list, axis=-1)
        coeffs[i] = pauli_string_coeff
    
    return coeffs, obs

In [4]:
#########################################################################################################

In [5]:
dist = 1.0
molecules = {
    "BeH2": "Be .0 .0 .0; H .0 .0 -" + str(dist) + "; H .0 .0 " + str(dist),
    "H2":"H .0 .0 .0; H .0 .0 " + str(dist),
    "LiH":"Li .0 .0 .0; H .0 .0 " + str(dist),
    "H2O": "H -0.0399 -0.0038 0.0; O 1.5780 0.8540 0.0; H 2.7909 -0.5159 0.0",
    "NH4": "N 0.0 0.0 0.149; H 0.0 0.947 -0.348; H  0.821 -0.474 -0.348; H -0.821 -0.474 -0.348"
}

In [6]:
# mol = 'H2'
# mol = 'LiH'
# mol = 'BeH2'
mol = "H2O"
# mol = "NH4"

In [7]:
converter = QubitConverter( JordanWignerMapper(), z2symmetry_reduction=None)
molecular_hamiltonian = MoleculeHamiltonian(molecules[mol] , converter=converter, basis='sto3g')
hamiltonian = molecular_hamiltonian.Hamiltonian( freeze_core = False )

driver = molecular_hamiltonian.driver
problem = molecular_hamiltonian.problem
n_qubits = hamiltonian.num_qubits

bodies = get_number_nbody_terms(hamiltonian)

In [8]:
circ_gs, eig_gs = molecular_hamiltonian.ComputeGroundState()
eig_gs

-79.59454251461285

In [9]:
total_shots = 100000
r = 40

In [10]:
# algorithm = ExpVal(n_shots = total_shots-1000,
#         bodies = bodies, 
#         r = 40, 
#         r_shots = 1000,
#         n_qubits = n_qubits)

# algorithm.get_interferences(circ_gs)

# coeffs, obs = get_obs_data(hamiltonian)

# exp_val = algorithm.exp_val(obs)


# results = np.sum(exp_val*coeffs).real

In [11]:
results = []

from tqdm import tqdm

for i in tqdm(range(20)):
    algorithm = ExpVal(n_shots = total_shots-1000,
            bodies = bodies, 
            r = r, 
            r_shots = 1000,
            n_qubits = n_qubits)

    algorithm.get_interferences(circ_gs)

    coeffs, obs = get_obs_data(hamiltonian)

    exp_val = algorithm.exp_val(obs)

    # print('Exp val calculado: ', np.sum(exp_val*coeffs).real)
    results.append(np.sum(exp_val*coeffs).real)

  5%|██▏                                         | 1/20 [00:16<05:17, 16.71s/it]

Exp val calculado:  -79.58656500821102


 10%|████▍                                       | 2/20 [00:34<05:11, 17.30s/it]

Exp val calculado:  -79.59550052679799


 15%|██████▌                                     | 3/20 [00:59<05:53, 20.81s/it]

...

Exp val calculado:  -79.59730961969048


 95%|████████████████████████████████████████▊  | 19/20 [06:23<00:19, 19.36s/it]

Exp val calculado:  -79.5920938755007


100%|███████████████████████████████████████████| 20/20 [06:49<00:00, 20.47s/it]

Exp val calculado:  -79.59674680243957


In [12]:
print(np.abs(results-eig_gs))

print(np.mean(np.abs(results-eig_gs)))

[0.00797751 0.00095801 0.00440247 0.02025423 0.00464473 0.00269704
 0.00680235 0.00768949 0.0014616  0.00251514 0.0216625  0.02703826
 0.00995118 0.01323477 0.00739765 0.02441412 0.01116187 0.00276711
 0.00244864 0.00220429]
0.009084147067560621


In [13]:
num_paulis = len(hamiltonian)
shots_runtime = total_shots//num_paulis
# shots_opflow = total_shots//num_paulis

In [14]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

#QiskitRuntimeService.save_account(channel="ibm_quantum", token="1a92dbd3ad8a3a75036e3b0fed7d2a10a0dfef2277960a619c033757aa1aa9bcc8f95922671a2fbd0b05339eea4aae6305bc43b5b2e6d33b161346f9983adf48")
service = QiskitRuntimeService(channel="ibm_quantum")

options = Options()
options.optimization_level = 1
options.resilience_level = 0
options.execution.shots = shots_runtime

# with Session(service=service, backend="ibmq_qasm_simulator") as session:
#     estimator = Estimator(session=session, options = options)

#     job = estimator.run(circ_gs, hamiltonian)

#     result = job.result()

# result_runtime = result.values[0]
# np.abs(result_runtime-eig_gs)

In [15]:
results_runtime = []

for i in tqdm(range(20)):


    with Session(service=service, backend="ibmq_qasm_simulator") as session:
        estimator = Estimator(session=session, options = options)

        job = estimator.run(circ_gs, hamiltonian)

        result = job.result()

    result_runtime = result.values[0]
    
    results_runtime.append(result_runtime)

100%|███████████████████████████████████████████| 20/20 [06:47<00:00, 20.36s/it]


In [16]:
print(np.abs(results_runtime-eig_gs))

print(np.mean(np.abs(results_runtime-eig_gs)))

[2.47009783e-01 1.52585621e-02 1.05789984e-01 3.15818306e-01
 1.75764024e-01 1.98398493e-05 6.16262380e-03 1.60827981e-01
 1.31260116e-01 3.77499885e-02 6.76411259e-02 1.56409133e-01
 4.05878066e-02 1.44858320e-01 3.44311800e-01 7.24070763e-02
 6.28773565e-02 4.13613509e-01 3.92615325e-01 1.29691666e-01]
0.1510337163644273


In [17]:
# from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, CircuitStateFn
# from qiskit import Aer
# from qiskit.utils import QuantumInstance

# backend = Aer.get_backend("aer_simulator")
# quantum_instance = QuantumInstance(backend, shots=shots_opflow)

# op = hamiltonian
# psi = CircuitStateFn(circ_gs)

# measurable_expression = StateFn(op, is_measurement=True).compose(psi)
# expectation = PauliExpectation().convert(measurable_expression)  
# sampler = CircuitSampler(quantum_instance).convert(expectation)

# result_opflow = sampler.eval().real
# np.abs(result_opflow-eig_gs)